In [1]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from gensim.models import Word2Vec
import numpy as np
from sklearn.preprocessing import MinMaxScaler
#####

/opt/homebrew/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


ImportError: cannot import name 'triu' from 'scipy.linalg' (/opt/homebrew/anaconda3/lib/python3.11/site-packages/scipy/linalg/__init__.py)

In [8]:
! pip install --upgrade scipy

In [2]:
df1 = pd.read_csv("Cleaned_Funding_Details.csv")
df1.head()

,Name,Funding Round,First Funding Date,Raised Amount,Valuation Amount,Investors
0,Raiinmaker,Seed,3 Apr 2024,$ 7.50M,NaN,"Jump Capital, Cypher Capital, MEXC, Gate.io"
1,Satoshisync,Undisclosed,3 Apr 2024,NaN,NaN,"ZBS CAPITAL, Maven Capital, MH Ventures, Chain..."
2,Legitimate,Seed,3 Apr 2024,$ 4.30M,NaN,"Lemniscap, Sfermion, Draper Associates, Verbal"
3,Mangata,Undisclosed,3 Apr 2024,$ 5.00M,$ 80M,"Polychain Capital, CMS Holdings, Master Ventur..."
4,Bonus-Block,Undisclosed,2 Apr 2024,$ 2.30M,NaN,"ZBS CAPITAL, Rarestone Capital, Faculty Group,..."


In [3]:
df2 = pd.read_csv("Cleaned_Overview_Details.csv")
df2.head()

,Crypto Name,Service,Total Raised,Description,Links
0,Raiinmaker,Blockchain Service,7500000.0,Raiinmaker is a project focused on acceleratin...,[[[['https://cryptorank.io/categories/blockcha...
1,Satoshisync,DeFi,4300000.0,SatoshiSync is The Permissionless Chain Agnost...,[[[['https://cryptorank.io/categories/blockcha...
2,Legitimate,Blockchain Infrastructure,10600000.0,Legitimate presents an on-chain framework meti...,[[[['https://cryptorank.io/categories/blockcha...
3,Mangata,Blockchain Infrastructure,300000.0,Gasp offers native cross-chain swaps without r...,"[[[['https://cryptorank.io/categories/chain'],..."
4,Bonus-Block,Blockchain Service,4000000.0,BonusBlock is a Web3 marketing platform that i...,"[[[['https://www.agora.finance/', 'https://twi..."


In [7]:
data = pd.merge(df1,df2, left_on = ["Name"], right_on = ["Crypto Name"])
data.head()

,Name,Funding Round,First Funding Date,Raised Amount,Valuation Amount,Investors,Crypto Name,Service,Total Raised,Description,Links
0,Raiinmaker,Seed,3 Apr 2024,$ 7.50M,NaN,"Jump Capital, Cypher Capital, MEXC, Gate.io",Raiinmaker,Blockchain Service,7500000.0,Raiinmaker is a project focused on acceleratin...,[[[['https://cryptorank.io/categories/blockcha...
1,Satoshisync,Undisclosed,3 Apr 2024,NaN,NaN,"ZBS CAPITAL, Maven Capital, MH Ventures, Chain...",Satoshisync,DeFi,4300000.0,SatoshiSync is The Permissionless Chain Agnost...,[[[['https://cryptorank.io/categories/blockcha...
2,Legitimate,Seed,3 Apr 2024,$ 4.30M,NaN,"Lemniscap, Sfermion, Draper Associates, Verbal",Legitimate,Blockchain Infrastructure,10600000.0,Legitimate presents an on-chain framework meti...,[[[['https://cryptorank.io/categories/blockcha...
3,Mangata,Undisclosed,3 Apr 2024,$ 5.00M,$ 80M,"Polychain Capital, CMS Holdings, Master Ventur...",Mangata,Blockchain Infrastructure,300000.0,Gasp offers native cross-chain swaps without r...,"[[[['https://cryptorank.io/categories/chain'],..."
4,Bonus-Block,Undisclosed,2 Apr 2024,$ 2.30M,NaN,"ZBS CAPITAL, Rarestone Capital, Faculty Group,...",Bonus-Block,Blockchain Service,4000000.0,BonusBlock is a Web3 marketing platform that i...,"[[[['https://www.agora.finance/', 'https://twi..."


In [10]:
data['First Funding Date'] = pd.to_datetime(data['First Funding Date'])

# Extract year, month, and day components
data['First Funding Year'] = data['First Funding Date'].dt.year
data['First Funding Month'] = data['First Funding Date'].dt.month
data['First Funding Day'] = data['First Funding Date'].dt.day

In [8]:
data['Valuation Amount'] = data['Valuation Amount'].str.extract(r'\$ (\d+\.?\d*)M').astype(float) * 1e6

In [59]:
processed_data = data.drop(columns=["Crypto Name", "Name", "Raised Amount", "First Funding Date", "Valuation Amount", "Links"])

In [60]:
processed_data.fillna("not available", inplace=True)
encoder = OneHotEncoder(sparse=False)
processed_data = pd.get_dummies(processed_data, columns= ["Service", "Funding Round"], dtype = int)
processed_data["Inverstors_and_desc"] = processed_data["Investors"]+" "+processed_data["Description"]
processed_data = processed_data.drop(columns=["Investors","Description"])
# Initialize the MinMaxScaler
scaler = MinMaxScaler()

# List of numeric column names
numeric_columns = ['Total Raised', 'First Funding Year', 'First Funding Month',
       'First Funding Day', 'Service_Blockchain Infrastructure',
       'Service_Blockchain Service', 'Service_CeFi', 'Service_Chain',
       'Service_DeFi', 'Service_GameFi', 'Service_Social',
       'Service_Stablecoin', 'Funding Round_Angel', 'Funding Round_Pre-Seed',
       'Funding Round_Pre-Series A', 'Funding Round_Seed',
       'Funding Round_Series A', 'Funding Round_Strategic',
       'Funding Round_Undisclosed']

# Apply min-max normalization to each numeric column
processed_data[numeric_columns] = scaler.fit_transform(processed_data[numeric_columns])
test_data = processed_data[-3:]
processed_data = processed_data[:-3]
processed_data.head()

,Total Raised,First Funding Year,First Funding Month,First Funding Day,Service_Blockchain Infrastructure,Service_Blockchain Service,Service_CeFi,Service_Chain,Service_DeFi,Service_GameFi,Service_Social,Service_Stablecoin,Funding Round_Angel,Funding Round_Pre-Seed,Funding Round_Pre-Series A,Funding Round_Seed,Funding Round_Series A,Funding Round_Strategic,Funding Round_Undisclosed,Inverstors_and_desc
0,0.087430,0.0,1.0,0.068966,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,"Jump Capital, Cypher Capital, MEXC, Gate.io Ra..."
1,0.049750,0.0,1.0,0.068966,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,"ZBS CAPITAL, Maven Capital, MH Ventures, Chain..."
2,0.123933,0.0,1.0,0.068966,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,"Lemniscap, Sfermion, Draper Associates, Verbal..."
3,0.002649,0.0,1.0,0.068966,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,"Polychain Capital, CMS Holdings, Master Ventur..."
4,0.046217,0.0,1.0,0.034483,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,"ZBS CAPITAL, Rarestone Capital, Faculty Group,..."


### RECOMMENDAR SYSTEM

### Weighted Similarity

# Streamlit App

In [ ]:
# Train Word2Vec model on textual descriptions
texts = [text.split() for text in processed_data['Inverstors_and_desc']]
embedding_dim = 100  # Adjust as needed
word2vec_model = Word2Vec(texts, vector_size=embedding_dim, window=5, min_count=1, workers=4)

# Create embeddings for textual descriptions
def create_embeddings(description):
    words = description.split()
    embeddings = [word2vec_model.wv[word] for word in words if word in word2vec_model.wv]
    if embeddings:
        return np.mean(embeddings, axis=0)
    else:
        return np.zeros(embedding_dim)

# Create feature matrix with embeddings and numerical features
feature_matrix = []
for index, entry in processed_data.iterrows():
    embeddings = create_embeddings(entry['Inverstors_and_desc'])
    numerical_features = [
        entry['Total Raised'],
        entry['First Funding Year'],
        entry['First Funding Month'],
        entry['First Funding Day'],
        entry['Service_Blockchain Infrastructure'],
        entry['Service_Blockchain Service'],
        entry['Service_CeFi'],
        entry['Service_Chain'],
        entry['Service_DeFi'],
        entry['Service_GameFi'],
        entry['Service_Social'],
        entry['Service_Stablecoin'],
        entry['Funding Round_Angel'],
        entry['Funding Round_Pre-Seed'],
        entry['Funding Round_Pre-Series A'],
        entry['Funding Round_Seed'],
        entry['Funding Round_Series A'],
        entry['Funding Round_Strategic'],
        entry['Funding Round_Undisclosed']
    ]
    feature_vector = np.concatenate([embeddings, numerical_features])
    feature_matrix.append(feature_vector)

feature_matrix = np.array(feature_matrix)
print(len(feature_matrix[0]))

# Perform SVD on the feature matrix
U, sigma, Vt = np.linalg.svd(feature_matrix)

# Determine the value of k based on explained variance ratio
explained_variance_ratio = np.cumsum(sigma ** 2) / np.sum(sigma ** 2)
threshold = 0.95  # Adjust as needed
k = np.argmax(explained_variance_ratio >= threshold) + 1

# Reconstruction of feature matrix
reconstructed_feature_matrix = np.dot(U[:, :k], np.dot(np.diag(sigma[:k]), Vt[:k, :]))

def match(test_data, processed_data, column_weights, top_n=5):
    all_top_indices = []
    all_top_similarities = []
    important_columns_per_match = []
    for index, entry in test_data.iterrows():
        embeddings = create_embeddings(entry['Inverstors_and_desc'])
        numerical_features = [
            entry['Total Raised'] * column_weights['Total Raised'],
            entry['First Funding Year'] * column_weights['First Funding Year'],
            entry['First Funding Month'] * column_weights['First Funding Month'],
            entry['First Funding Day'] * column_weights['First Funding Day'],
            entry['Service_Blockchain Infrastructure'] * column_weights['Service_Blockchain Infrastructure'],
            entry['Service_Blockchain Service'] * column_weights['Service_Blockchain Service'],
            entry['Service_CeFi'] * column_weights['Service_CeFi'],
            entry['Service_Chain'] * column_weights['Service_Chain'],
            entry['Service_DeFi'] * column_weights['Service_DeFi'],
            entry['Service_GameFi'] * column_weights['Service_GameFi'],
            entry['Service_Social'] * column_weights['Service_Social'],
            entry['Service_Stablecoin'] * column_weights['Service_Stablecoin'],
            entry['Funding Round_Angel'] * column_weights['Funding Round_Angel'],
            entry['Funding Round_Pre-Seed'] * column_weights['Funding Round_Pre-Seed'],
            entry['Funding Round_Pre-Series A'] * column_weights['Funding Round_Pre-Series A'],
            entry['Funding Round_Seed'] * column_weights['Funding Round_Seed'],
            entry['Funding Round_Series A'] * column_weights['Funding Round_Series A'],
            entry['Funding Round_Strategic'] * column_weights['Funding Round_Strategic'],
            entry['Funding Round_Undisclosed'] * column_weights['Funding Round_Undisclosed']
        ]
        feature_vector = np.concatenate([embeddings, numerical_features])
        
        # Calculate similarities between the new data and existing data
        similarities = np.dot(reconstructed_feature_matrix, feature_vector.T)
        
        # Find top n similar services
        top_indices = np.argsort(similarities)[::-1][:top_n]
        top_similarities = similarities[top_indices]
        
        all_top_indices.extend(top_indices)
        all_top_similarities.extend(top_similarities)

    # Find top n unique indices based on the top n similarity scores
    sorted_indices = np.argsort(all_top_similarities)[::-1]
    unique_top_indices = [all_top_indices[idx] for idx in sorted_indices][:top_n]
    
    return unique_top_indices

# Define column weights
column_weights = {
    'Total Raised': 0.8,
    'First Funding Year': 0.6,
    'First Funding Month': 0.6,
    'First Funding Day': 0.6,
    'Service_Blockchain Infrastructure': 1.0,
    'Service_Blockchain Service': 1.0,
    'Service_CeFi': 1.0,
    'Service_Chain': 1.0,
    'Service_DeFi': 1.0,
    'Service_GameFi': 1.0,
    'Service_Social': 1.0,
    'Service_Stablecoin': 1.0,
    'Funding Round_Angel': 1.0,
    'Funding Round_Pre-Seed': 1.0,
    'Funding Round_Pre-Series A': 1.0,
    'Funding Round_Seed': 1.0,
    'Funding Round_Series A': 1.0,
    'Funding Round_Strategic': 1.0,
    'Funding Round_Undisclosed': 1.0
}

# Call the match function with column weights
# print("Top 5 Matches for the new data (indices):", top_indices)


In [ ]:
import streamlit as st
import pandas as pd

def main():
    st.title("Start Up Recommendation System")

    # Get user input
    user_id = st.number_input("How many start-ups should we recommend to you?", min_value=1, max_value=10, value=1, step=1)

    # Get recommendations for the user
    top_indices = match(test_data, processed_data, column_weights, top_n=top_n)

    # Display the recommendations
    st.write("### Recommended Movies:")
    for start_ups in top_indices:
        data.iloc[top_indices]
        st.write(f"- { data.iloc[top_indices]}")

if __name__ == "__main__":
    main()